# A LSTM for Part-of-Speech Tagging



In [1]:
%matplotlib inline

## Sequence Models and Long-Short Term Memory Networks


At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding hidden state $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.






## LSTM's in Pytorch


Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.

In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Part-of-Speech Tagging


In this section, we will use an LSTM to get part of speech tags.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocabulary. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.





## Prepare data:

In [3]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

Print the structure of the training data

In [4]:
print(training_data)
print(training_data[0])
print(training_data[0][0])

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']), (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]
(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN'])
['The', 'dog', 'ate', 'the', 'apple']


In [5]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}



{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [6]:
ix_to_word = {}
for i, word in enumerate(tag_to_ix):
    ix_to_word[i] = word

print(ix_to_word)

{0: 'DET', 1: 'NN', 2: 'V'}


In [7]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 10

## Create the model:



A few cells to understand and test the LSTM

### Question: explain carefully the following cell exploiting a LSTM.

In [8]:
vocab_size = len(word_to_ix)
print(vocab_size)
embedding_dim = EMBEDDING_DIM
hidden_dim = HIDDEN_DIM
word_embeddings = nn.Embedding(vocab_size, embedding_dim)
print(word_embeddings)


9
Embedding(9, 6)


In [9]:
sentence = training_data[0][0]
print(sentence)
sentence_in = prepare_sequence(sentence, word_to_ix)
print(sentence_in)



['The', 'dog', 'ate', 'the', 'apple']
tensor([0, 1, 2, 3, 4])


In [10]:
embeds = word_embeddings(sentence_in)
print(embeds)
print(embeds.shape)
embeds_view = embeds.view(len(sentence), 1, -1)
print(embeds_view)
print(embeds_view.shape)

tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092],
        [-0.9798, -1.6091, -0.7121,  0.3037, -0.7773, -0.2515],
        [-0.2223,  1.6871,  0.2284,  0.4676, -0.6970, -1.1608],
        [ 0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073],
        [ 1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177]],
       grad_fn=<EmbeddingBackward>)
torch.Size([5, 6])
tensor([[[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092]],

        [[-0.9798, -1.6091, -0.7121,  0.3037, -0.7773, -0.2515]],

        [[-0.2223,  1.6871,  0.2284,  0.4676, -0.6970, -1.1608]],

        [[ 0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073]],

        [[ 1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177]]],
       grad_fn=<ViewBackward>)
torch.Size([5, 1, 6])


In [11]:
lstm = nn.LSTM(embedding_dim, hidden_dim)
# Parameters of LSTM
# input_size – The number of expected features in the input x
# hidden_size – The number of features in the hidden state h
lstm_out, _ = lstm(embeds_view)
# LSTM input of shape (seq_len, batch, input_size): 
#tensor containing the features of the input sequence. 
# 
# LSTM output of shape (seq_len, batch, num_directions * hidden_size): 
# tensor containing the output features (h_t) from the last layer of the LSTM, for each t.
print(lstm_out)
print(lstm_out.shape)

tensor([[[ 1.9225e-02,  4.1500e-02, -2.4441e-01, -7.7552e-02, -1.7128e-02,
           1.2690e-01, -5.9875e-02, -1.2850e-01, -1.1659e-03,  6.6426e-02]],

        [[ 4.2322e-02, -5.6735e-02, -1.9042e-01, -4.6320e-02, -1.3861e-01,
           6.1933e-02, -2.7061e-02, -1.1546e-01,  9.3254e-02,  2.6355e-02]],

        [[-3.2475e-04,  1.2914e-01, -3.5927e-01,  2.2265e-02,  4.1101e-02,
           2.6102e-01,  2.4373e-01, -2.1269e-01, -8.4736e-02, -4.4971e-02]],

        [[-7.5424e-02,  1.6407e-01, -1.4608e-01,  1.2357e-01,  1.2083e-01,
          -4.0878e-02,  1.6828e-01, -1.4338e-01, -8.0442e-02, -1.2277e-01]],

        [[ 1.1725e-01,  1.9175e-01, -1.8829e-01, -6.5649e-02,  1.7355e-01,
           1.0598e-01, -1.5585e-02, -4.9462e-02,  1.8246e-01, -1.8868e-03]]],
       grad_fn=<StackBackward>)
torch.Size([5, 1, 10])


### Question: add a LSTM layer in the following neural network.

Just look at the code.

### Question: what is the role of the linear layer in the following neural network?

We add a linear layer for tagging the word (classification task).

### Question: how to interpret the output of the "forward" function?

For each word, it gives the values of the possible tags. Hence, there are 3 values. The maximum value corresponds to the most likely tag.

In [12]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

## Train the model:



In [13]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [14]:
def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

### Question: compute and print the tags before the training.

In [15]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(training_data[0][0])
    print(inputs)
    print(tag_scores)
    for i, tags in enumerate(tag_scores):
        print('Prediction: {}'.format(get_max_prob_result(tags, ix_to_word)))


['The', 'dog', 'ate', 'the', 'apple']
tensor([0, 1, 2, 3, 4])
tensor([[-0.7634, -1.2999, -1.3418],
        [-0.7818, -1.2551, -1.3573],
        [-0.8378, -1.2004, -1.3233],
        [-0.8355, -1.2462, -1.2775],
        [-0.8350, -1.2380, -1.2867]])
Prediction: DET
Prediction: DET
Prediction: DET
Prediction: DET
Prediction: DET


In [16]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

### Question: compute and print the tags after the training.

In [17]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

    for i, tags in enumerate(tag_scores):
      print('Prediction: {}'.format(get_max_prob_result(tags, ix_to_word)))


tensor([[-0.0234, -3.8914, -5.9213],
        [-4.5506, -0.0148, -5.4778],
        [-4.7502, -4.7769, -0.0172],
        [-0.0210, -4.4648, -4.6791],
        [-4.6447, -0.0161, -5.0580]])
Prediction: DET
Prediction: NN
Prediction: V
Prediction: DET
Prediction: NN
